In [14]:
import pandas as pd
DATASET = "amazon_reviews_us_Grocery_v1_00.tsv"
import logging
import warnings
import sys
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS
from nltk.corpus import wordnet
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer

warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
logging.disable(sys.maxsize)



In [183]:
dataset = pd.read_table(DATASET, error_bad_lines=False, header=0, warn_bad_lines=False) 
# there are some malformed entries the dataset... let's ignore those for now


In [184]:
len(dataset.index)

2393379

In [185]:
synonyms = []
for subject in subjects:
    for syn in wordnet.synsets(subject):
        for l in syn.lemmas():
            synonyms.append(l.name())

print(set(synonyms))


NameError: name 'subjects' is not defined

In [186]:
### Let's see what the dataset looks like... 
dataset.head()


,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,42521656,R26MV8D0KG6QI6,B000SAQCWC,159713740,"The Cravings Place Chocolate Chunk Cookie Mix, 23-Ounce Bags (Pack of 6)",Grocery,5,0.0,0.0,N,Y,Using these for years - love them.,"As a family allergic to wheat, dairy, eggs, nuts, and several other things, we love the entire Cravings Place line of products as it allows us to bake treats with minimal effort and ingredients. Most allergy-free and gluten-free mixes usually just omit one or two allergens at most, so it's great to see a mix created without many of the most common allergens. (Note these still have soy and corn). We consume these on a regular basis and have been doing so for years.",2015-08-31
1,US,12049833,R1OF8GP57AQ1A0,B00509LVIQ,138680402,"Mauna Loa Macadamias, 11 Ounce Packages",Grocery,5,0.0,0.0,N,Y,Wonderful,"My favorite nut. Creamy, crunchy, salty, and slightly sweet - what more could you ask for?",2015-08-31
2,US,107642,R3VDC1QB6MC4ZZ,B00KHXESLC,252021703,Organic Matcha Green Tea Powder - 100% Pure Matcha (No Sugar Added - Unsweetened Pure Green Tea - No Coloring Added Like Others) 4oz,Grocery,5,0.0,0.0,N,N,Five Stars,This green tea tastes so good! My girlfriend loves it too.,2015-08-31
3,US,6042304,R12FA3DCF8F9ER,B000F8JIIC,752728342,15oz Raspberry Lyons Designer Dessert Syrup Sauce,Grocery,5,0.0,0.0,N,Y,Five Stars,I love Melissa's brand but this is a great second when I can't get Melissa's brand.,2015-08-31
4,US,18123821,RTWHVNV6X4CNJ,B004ZWR9RQ,552138758,"Stride Spark Kinetic Fruit Sugar Free Gum, 14-Count (Pack of 12)",Grocery,5,0.0,0.0,N,Y,Five Stars,good,2015-08-31


In [187]:
#lots of nan values... :( 
dataset.isnull().sum()

marketplace           0
customer_id           0
review_id             0
product_id            0
product_parent        0
product_title         0
product_category      0
star_rating          22
helpful_votes        23
total_votes          23
vine                 23
verified_purchase    23
review_headline      34
review_body          88
review_date          65
dtype: int64

In [188]:
# Counts of reviews for same product
dataset['product_id'].value_counts()

B00DS842HS    9998
B007Y59HVM    4526
B00M2OGS08    3881
B007TGDXMU    3691
B0029XDZIK    3461
              ... 
B00AQKFQXU       1
B000LRIGUS       1
B00HUHVS3E       1
B00P4CD4KC       1
B00J3YSH40       1
Name: product_id, Length: 305026, dtype: int64

In [189]:
# Counts of reviews with same body
dataset['review_body'].value_counts()

Great                                                                                                                                                                                                                                                                                                                                                                                                                                       3592
good                                                                                                                                                                                                                                                                                                                                                                                                                                        3479
great                                                                                                                                 

In [190]:
# Counts of reviews with same headline
dataset['review_headline'].value_counts()

Five Stars                                                                                                335898
Four Stars                                                                                                 46683
Three Stars                                                                                                22617
One Star                                                                                                   19808
Two Stars                                                                                                  12316
                                                                                                           ...  
Eh....save your money and make your own.                                                                       1
Just chili beans, nothing too exciting, consistently good quality.                                             1
Love this matcha green tea!                                                                     

In [191]:
# Lots of spammy duplicate reviews...might be worth checking if the same person is duplicating the reviews across multiple products
x = dataset[dataset['review_body'].duplicated() == True]
x.head()
print(len(x.index))

175162


In [192]:
#remove NaNs from the dataset  for now...
dataset = dataset.dropna()

In [201]:
dataset["n_reviews"] = dataset.groupby('product_id')['product_id'].transform('count')

In [202]:
dataset.head()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,n_reviews
0,US,42521656,R26MV8D0KG6QI6,B000SAQCWC,159713740,"The Cravings Place Chocolate Chunk Cookie Mix, 23-Ounce Bags (Pack of 6)",Grocery,5,0.0,0.0,N,Y,Using these for years - love them.,"As a family allergic to wheat, dairy, eggs, nuts, and several other things, we love the entire Cravings Place line of products as it allows us to bake treats with minimal effort and ingredients. Most allergy-free and gluten-free mixes usually just omit one or two allergens at most, so it's great to see a mix created without many of the most common allergens. (Note these still have soy and corn). We consume these on a regular basis and have been doing so for years.",2015-08-31,12
1,US,12049833,R1OF8GP57AQ1A0,B00509LVIQ,138680402,"Mauna Loa Macadamias, 11 Ounce Packages",Grocery,5,0.0,0.0,N,Y,Wonderful,"My favorite nut. Creamy, crunchy, salty, and slightly sweet - what more could you ask for?",2015-08-31,473
2,US,107642,R3VDC1QB6MC4ZZ,B00KHXESLC,252021703,Organic Matcha Green Tea Powder - 100% Pure Matcha (No Sugar Added - Unsweetened Pure Green Tea - No Coloring Added Like Others) 4oz,Grocery,5,0.0,0.0,N,N,Five Stars,This green tea tastes so good! My girlfriend loves it too.,2015-08-31,302
3,US,6042304,R12FA3DCF8F9ER,B000F8JIIC,752728342,15oz Raspberry Lyons Designer Dessert Syrup Sauce,Grocery,5,0.0,0.0,N,Y,Five Stars,I love Melissa's brand but this is a great second when I can't get Melissa's brand.,2015-08-31,24
4,US,18123821,RTWHVNV6X4CNJ,B004ZWR9RQ,552138758,"Stride Spark Kinetic Fruit Sugar Free Gum, 14-Count (Pack of 12)",Grocery,5,0.0,0.0,N,Y,Five Stars,good,2015-08-31,17


In [203]:
#nltk.download('vader_lexicon')

def sentiment_heuristic(row):
    return sia.polarity_scores(row)['compound']


In [ ]:
# This will take ages to run lol
#dataset['sentiment_score'] = dataset["review_body"].apply(sentiment_heuristic)

In [204]:
dataset.head()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,n_reviews
0,US,42521656,R26MV8D0KG6QI6,B000SAQCWC,159713740,"The Cravings Place Chocolate Chunk Cookie Mix, 23-Ounce Bags (Pack of 6)",Grocery,5,0.0,0.0,N,Y,Using these for years - love them.,"As a family allergic to wheat, dairy, eggs, nuts, and several other things, we love the entire Cravings Place line of products as it allows us to bake treats with minimal effort and ingredients. Most allergy-free and gluten-free mixes usually just omit one or two allergens at most, so it's great to see a mix created without many of the most common allergens. (Note these still have soy and corn). We consume these on a regular basis and have been doing so for years.",2015-08-31,12
1,US,12049833,R1OF8GP57AQ1A0,B00509LVIQ,138680402,"Mauna Loa Macadamias, 11 Ounce Packages",Grocery,5,0.0,0.0,N,Y,Wonderful,"My favorite nut. Creamy, crunchy, salty, and slightly sweet - what more could you ask for?",2015-08-31,473
2,US,107642,R3VDC1QB6MC4ZZ,B00KHXESLC,252021703,Organic Matcha Green Tea Powder - 100% Pure Matcha (No Sugar Added - Unsweetened Pure Green Tea - No Coloring Added Like Others) 4oz,Grocery,5,0.0,0.0,N,N,Five Stars,This green tea tastes so good! My girlfriend loves it too.,2015-08-31,302
3,US,6042304,R12FA3DCF8F9ER,B000F8JIIC,752728342,15oz Raspberry Lyons Designer Dessert Syrup Sauce,Grocery,5,0.0,0.0,N,Y,Five Stars,I love Melissa's brand but this is a great second when I can't get Melissa's brand.,2015-08-31,24
4,US,18123821,RTWHVNV6X4CNJ,B004ZWR9RQ,552138758,"Stride Spark Kinetic Fruit Sugar Free Gum, 14-Count (Pack of 12)",Grocery,5,0.0,0.0,N,Y,Five Stars,good,2015-08-31,17


In [205]:

def filter_bad_star(row):
    '''
    determine if a row is a negative review based on its score. Potentially extend it to include sentiment analysis 
    and presence of specific negative words 
    '''
    return row["star_rating"].astype(int) <=3

def filter_good_star(row):
    '''
    determine if a row is a positive review based on its score.
    '''
    return row["star_rating"].astype(int) > 3

In [221]:
negative_rows = dataset.loc[filter_bad_star]
negative_rows.head()
len(negative_rows.index)

445115

In [222]:
negative_rows.head()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,n_reviews
5,US,23649464,RIG9AWFOGRDVO,B00AL6QBZ6,681475449,Herr's Popcorn Hot Cheese 1 Oz (Pack of 30),Grocery,2,1.0,1.0,N,Y,Not Happy,The popcorn was stale.,2015-08-31,8
9,US,19624355,R1ODXB3C9UP3NL,B00J074W94,2499702,"Orgain Organic Plant Based Protein Powder, Packaging May Vary",Grocery,1,1.0,3.0,N,N,Disgusting now and difficult on digestion,Used to be a decent product. Disgusting now and difficult on digestion. All 3 purchased from Costco over past couple months end in same result -- open the container and it smells like rancid oil. Something not right about how they are making/processing this powder now. Will not buy again.,2015-08-31,522
17,US,22765168,R3T6TTD2IN0EFZ,B00XDXMLL2,971154239,"Skippy Creamy Peanut Butter, with Salted Caramel 15oz (Pack of 4)",Grocery,1,4.0,4.0,N,N,"1 Out Of 5 Of My Co-Workers Thought It Was ""Okay""","I bought this from a local super market on a whim and decided to let people know how it tastes. I'm a huge fan of peanut butter and salted caramel.<br /><br />For instance, I had a Salted Caramel and Almond Kind Bar today and it was amazing. It tasted like you would expect it to taste.<br /><br />However, this particular product tastes like a chemical spill. It starts off with a peanut butter taste, but then the (caramel I'm guessing) tastes kind of burnt and chemical-like, and then it finishes with a very salty burnt taste.<br /><br />I had some on a spoon and disliked it, then I put some on pretzel bread and it was slightly palatable, but overall I would avoid this product as it doesn't really taste like salted caramel peanut butter.<br /><br />Just for fun, I let my co-workers taste it and only 1 out of 5 of them thought that this product tasted &#34;okay&#34;. The rest of them shared my sentiment.",2015-08-31,6
23,US,35636887,R9MISLBRG08FX,B00DBSFXUA,294404974,"Keebler Town House Pita Crackers, 9.5 Ounce",Grocery,1,0.0,0.0,N,Y,pita crackers,not craze about these. nothing really wrong with them just no into them. These crackers are small so not much room for cheese.,2015-08-31,107
26,US,12650237,R2A9O8CWZ1PP74,B0083GJKR2,868929824,"Eclipse Sugar Free Gum, Spearmint, 120 Piece Bottle",Grocery,3,0.0,0.0,N,Y,Three Stars,it's gum..,2015-08-31,25


In [223]:
positive_rows = dataset.loc[filter_good_star]
positive_rows.head()
len(positive_rows.index)

1948123

In [224]:
bad_words = [
    "stinky",
    "stank"
    "moldy",
    "mouldy",
    "rotted",
    "rotten",
    "stale"
    "stinking",
    "icky",
    "curdled",
    "danger",
    "dangerous",
    "ammonia",
    "unsafe",
    "avoid"
    "sick"
]

def filter_bad_words(row):
    '''
    determine if a row is a negative review based on its words.
    '''
    return row["review_headline"].str.contains("\\b(%s)\\b" % '|'.join(bad_words))


In [225]:
negative_rows = negative_rows.loc[filter_bad_words]
len(negative_rows.index)

379

In [226]:
negative_rows['sentiment_score'] = negative_rows["review_body"].apply(sentiment_heuristic)

In [227]:
negative_rows.head()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,n_reviews,sentiment_score
8911,US,37307498,RZP5EWKOBX5KH,B009L9TMBY,696947556,"Deluxe Mixed Nuts Raw, 2Lbs",Grocery,2,1.0,1.0,N,Y,Ixnay on the rotten nuts.,"Probably would have been good, but my nuts were rancid.",2015-08-28,20,-0.7750
22258,US,47973491,R1NOLST0E8IATT,B0036ZIF9Y,923729241,Gemlik Type Black Olives LUX – 1.1lb (500g),Grocery,1,2.0,2.0,N,Y,Mine were rotten and tasted horrible...,"I am Turkish and I've never had any problems with Marmarabirlik olives in my whole life. HOWEVER, this is the first time I've ordered them online and I have to say there were the worst olives I've ever tasted. They were rotten! My Mom even commented that these olives are bad. Maybe I got a bad batch because this company makes one of the very best Turkish olives on the market. I'm extremely disappointed with this online order.",2015-08-24,7,-0.9069
35555,US,30293525,R1FRDE0OIA546B,B00TCOAXR0,915238907,Chung Jung One Anchovy Sauce Blue,Grocery,1,3.0,9.0,N,Y,.It tastes like it was made with rotted anchovies....,The taste is okay....Nothing spectacular....It tastes like it was made with rotted anchovies....<br />The main problem: I braised something using this sauce and my apartment stunk like rotted fish for days....<br />The cheap plastic bottle does not make this product more enticing.....,2015-08-19,3,-0.0772
36783,US,50106447,R2KIR9GSCGNQI6,B002N723Q2,479009963,"Numi Organic Black Tea, Loose Leaf Tea",Grocery,1,1.0,3.0,N,Y,"Good taste, but with rotten flavor","Good taste, but absolutely no flavor, both dry and brewed. Contacted Numi Customer Service. Got careless response, They sent me a couple of tea bags which were much worse than the loose Emperor's Puerh tea. After a month or so it smells rotten. Still taste is OK",2015-08-19,15,-0.9272
54550,US,22365141,RGSXU84MSLDY7,B006GL6N9E,776164341,Fruit Basket Delight with Cheese and Nuts,Grocery,2,0.0,0.0,N,Y,rotten fruit,I sent this to my Mom for Mothers Day and the fruit was rotten! It made me feel terrible.,2015-08-12,159,-0.7712


In [232]:
negative_rows = negative_rows[negative_rows['sentiment_score'] < 0]

In [233]:
sorted_negative_rows = negative_rows.sort_values("product_id")
negative_rows.head(n=8)

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,n_reviews,sentiment_score
8911,US,37307498,RZP5EWKOBX5KH,B009L9TMBY,696947556,"Deluxe Mixed Nuts Raw, 2Lbs",Grocery,2,1.0,1.0,N,Y,Ixnay on the rotten nuts.,"Probably would have been good, but my nuts were rancid.",2015-08-28,20,-0.7750
22258,US,47973491,R1NOLST0E8IATT,B0036ZIF9Y,923729241,Gemlik Type Black Olives LUX – 1.1lb (500g),Grocery,1,2.0,2.0,N,Y,Mine were rotten and tasted horrible...,"I am Turkish and I've never had any problems with Marmarabirlik olives in my whole life. HOWEVER, this is the first time I've ordered them online and I have to say there were the worst olives I've ever tasted. They were rotten! My Mom even commented that these olives are bad. Maybe I got a bad batch because this company makes one of the very best Turkish olives on the market. I'm extremely disappointed with this online order.",2015-08-24,7,-0.9069
35555,US,30293525,R1FRDE0OIA546B,B00TCOAXR0,915238907,Chung Jung One Anchovy Sauce Blue,Grocery,1,3.0,9.0,N,Y,.It tastes like it was made with rotted anchovies....,The taste is okay....Nothing spectacular....It tastes like it was made with rotted anchovies....<br />The main problem: I braised something using this sauce and my apartment stunk like rotted fish for days....<br />The cheap plastic bottle does not make this product more enticing.....,2015-08-19,3,-0.0772
36783,US,50106447,R2KIR9GSCGNQI6,B002N723Q2,479009963,"Numi Organic Black Tea, Loose Leaf Tea",Grocery,1,1.0,3.0,N,Y,"Good taste, but with rotten flavor","Good taste, but absolutely no flavor, both dry and brewed. Contacted Numi Customer Service. Got careless response, They sent me a couple of tea bags which were much worse than the loose Emperor's Puerh tea. After a month or so it smells rotten. Still taste is OK",2015-08-19,15,-0.9272
54550,US,22365141,RGSXU84MSLDY7,B006GL6N9E,776164341,Fruit Basket Delight with Cheese and Nuts,Grocery,2,0.0,0.0,N,Y,rotten fruit,I sent this to my Mom for Mothers Day and the fruit was rotten! It made me feel terrible.,2015-08-12,159,-0.7712
61021,US,14391210,RMC6ESIHDJHO8,B009XRHRZC,756206582,Dutch Soft Licorice Clogs-(Klompjes Drop)-GLUTEN FREE-4oz bag,Grocery,1,0.0,0.0,N,Y,sort of like eating some Brie cheeses that waft ammonia up the ...,"I can taste an ammonia after taste with this licorice. So, so gross, sort of like eating some Brie cheeses that waft ammonia up the sinuses. I am so disappointed because it is difficult to find gluten free licorice (black is real, the red stuff is just candy). The one shaped like cats is thinner and did NOT have that aftertaste.",2015-08-10,1,-0.5470
70811,US,13861214,R1X6047VQJ7JQ7,B00T77QT5M,290213412,"Barnana Organic Chewy Banana Bites Variety Pack, 12 Count",Grocery,1,0.0,2.0,N,Y,These taste like rotten moldy bananas,"These taste like rotten moldy bananas. I was trying to find a product that came close to the banana chunks that the Philippine brand made since I can't seem to find them anymore, but these aren't even edible much less comparible.<br />Unfortunately it won't let me give no star.",2015-08-07,3,-0.4939
81129,US,13403681,R1AY63RMD0KFBT,B00SHR75HO,697739221,Just Add Coffee Protein Creamer - Nonfat & No Added Sugar.,Grocery,2,1.0,1.0,N,Y,I didn't mind the taste but It curdled when i ...,"I didn't mind the taste but It curdled when i mixed it with my coffee, so gross! The expiration date was for a year from now so not sure what is in this but I will pass.",2015-08-04,14,-0.8638


In [231]:
len(negative_rows.index)

235

In [236]:
negative_rows["n_bad_reviews"] = negative_rows.groupby('product_id')['product_id'].transform('count')
negative_rows["bad_review_percent"] = negative_rows["n_bad_reviews"] / negative_rows["n_reviews"]

In [237]:
negative_rows.head()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,n_reviews,sentiment_score,n_bad_reviews,bad_review_percent
8911,US,37307498,RZP5EWKOBX5KH,B009L9TMBY,696947556,"Deluxe Mixed Nuts Raw, 2Lbs",Grocery,2,1.0,1.0,N,Y,Ixnay on the rotten nuts.,"Probably would have been good, but my nuts were rancid.",2015-08-28,20,-0.7750,1,0.050000
22258,US,47973491,R1NOLST0E8IATT,B0036ZIF9Y,923729241,Gemlik Type Black Olives LUX – 1.1lb (500g),Grocery,1,2.0,2.0,N,Y,Mine were rotten and tasted horrible...,"I am Turkish and I've never had any problems with Marmarabirlik olives in my whole life. HOWEVER, this is the first time I've ordered them online and I have to say there were the worst olives I've ever tasted. They were rotten! My Mom even commented that these olives are bad. Maybe I got a bad batch because this company makes one of the very best Turkish olives on the market. I'm extremely disappointed with this online order.",2015-08-24,7,-0.9069,1,0.142857
35555,US,30293525,R1FRDE0OIA546B,B00TCOAXR0,915238907,Chung Jung One Anchovy Sauce Blue,Grocery,1,3.0,9.0,N,Y,.It tastes like it was made with rotted anchovies....,The taste is okay....Nothing spectacular....It tastes like it was made with rotted anchovies....<br />The main problem: I braised something using this sauce and my apartment stunk like rotted fish for days....<br />The cheap plastic bottle does not make this product more enticing.....,2015-08-19,3,-0.0772,1,0.333333
36783,US,50106447,R2KIR9GSCGNQI6,B002N723Q2,479009963,"Numi Organic Black Tea, Loose Leaf Tea",Grocery,1,1.0,3.0,N,Y,"Good taste, but with rotten flavor","Good taste, but absolutely no flavor, both dry and brewed. Contacted Numi Customer Service. Got careless response, They sent me a couple of tea bags which were much worse than the loose Emperor's Puerh tea. After a month or so it smells rotten. Still taste is OK",2015-08-19,15,-0.9272,1,0.066667
54550,US,22365141,RGSXU84MSLDY7,B006GL6N9E,776164341,Fruit Basket Delight with Cheese and Nuts,Grocery,2,0.0,0.0,N,Y,rotten fruit,I sent this to my Mom for Mothers Day and the fruit was rotten! It made me feel terrible.,2015-08-12,159,-0.7712,1,0.006289


In [241]:
# Reviews sorted by proportion of reviews that are 'bad'
negative_rows.sort_values(by=['bad_review_percent', 'n_reviews'], ascending=False)

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,n_reviews,sentiment_score,n_bad_reviews,bad_review_percent
61021,US,14391210,RMC6ESIHDJHO8,B009XRHRZC,756206582,Dutch Soft Licorice Clogs-(Klompjes Drop)-GLUTEN FREE-4oz bag,Grocery,1,0.0,0.0,N,Y,sort of like eating some Brie cheeses that waft ammonia up the ...,"I can taste an ammonia after taste with this licorice. So, so gross, sort of like eating some Brie cheeses that waft ammonia up the sinuses. I am so disappointed because it is difficult to find gluten free licorice (black is real, the red stuff is just candy). The one shaped like cats is thinner and did NOT have that aftertaste.",2015-08-10,1,-0.5470,1,1.000000
110798,US,30804716,R3AS6OXW82P248,B00VN76PCQ,584830304,"Crunchies Freeze Dried Fruit Snack Organic Variety Pack, 5 Count (Pack of 6)",Grocery,3,0.0,1.0,N,Y,Disappointing and dangerous for toddlers,"These were rather disappointing - they are very tiny bits of freeze dried fruit in a very small package. The mango and apple bits are very hard, even when soaked in milk (I use this type of fruit in cereal). I definitely would not give these to toddlers because of danger of choking on the hard ones.",2015-07-24,1,-0.8437,1,1.000000
354911,US,32360043,R1DIPL5U42RRC3,B00BNI7GFK,30184486,Sour Neon Gummy Worms 5 pounds Bulk Bag,Grocery,1,1.0,1.0,N,Y,but these things were terrible! Just icky sugar,"Sorry, but these things were terrible! Just icky sugar, not even sour. I got rid of the stuff",2015-04-29,1,-0.6800,1,1.000000
595809,US,37764355,R17CMHZXMG73ED,B00CMNGC1Y,995975178,4C Drink Mix - Wild Berry Pom - 16qt (Pack of 4),Grocery,2,0.0,1.0,N,Y,Tastes good but has fake sweetener in it. Acesulfame! this stuff is dangerous and makes my heart flutter and race.,This product contains acesulfame sweetener which gives me serious heart flutter and raciness. acesulfame is made in lab and fake sugar. I wish 4c would just use regular sugar. These fake sweeteners come at a price not yet fully revealed. 20 years of drinking aspartame or acesulfame could end up causing serious health problems down the road. Not worth the zero calorie. If it makes my heart race and flutter after a few drinks this proves that stuff has side effects. Luckily for me it's instant. What about those who show no signs now? Will they end up with cancer in 20 years?,2015-02-13,1,-0.9027,1,1.000000
686804,US,1930338,R1FN4ABM0VWLCZ,B00J2UY4R4,491138391,"Madhava AgaveFIVE Drink Mix, 6 Count (Pack of 12)",Grocery,1,0.0,0.0,N,N,"Smells amazing, tastes like stinky feet.",Don't buy this stuff. I took one sip and spit it in the sink. How did this even make it past consumer testing? This is terrible. It tastes like crushed powdered medicine. Bleh!,2015-01-13,1,-0.5707,1,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
847293,US,5810748,R1LT7LRGCEZMY6,B000CR1ELU,306270194,Premium Madagascar Vanilla Beans 1/4 lb (26-30 beans) JR Mushrooms Brand,Grocery,1,2.0,3.0,N,Y,moldy/rotten,"My first shipment was great, but this shipment, 95% of the vanilla beans were useless and rotted/moldy. Not fermented, I mean bad, off. I was very disappointed and couldn't get amazon or the vendor to do much as it seems it's very hard to make a claim these days. They just lost my business buying vanilla beans in bulk. Too bad, b/c I need to buy half a pound of beans.",2014-11-21,630,-0.6067,1,0.001587
649016,US,15005427,RNDLJRATCWC9F,B0008D6WBA,356715615,"Great Lakes Unflavored Gelatin, Kosher, 16 Ounce Can",Grocery,2,0.0,0.0,N,Y,EW-rotten flesh smell and taste,It has a rotten flesh smell/taste to it that makes me gag. I am going back to Knox. EW.,2015-01-26,739,-0.6908,1,0.001353
2032911,US,25568272,R2NSMM7PGKFFGO,B0046HJN5G,840202637,KIND Minis Variety Pack,Grocery,2.0,2.0,3.0,N,Y,"Love the bars, but they made a dangerous change to the items included in this mix..

In [54]:
negative_rows['product_id'].value_counts()
negative_rows['product_parent'].value_counts()

572710338    3
23413404     2
969947869    2
92995222     2
162663470    2
            ..
705473714    1
323988147    1
340036861    1
122523830    1
127590911    1
Name: product_parent, Length: 220, dtype: int64

In [57]:
negative_rows.apply(pd.value_counts)

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,sentiment_score
-0.9843,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
-0.9787,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
-0.9771,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
-0.9741,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
-0.9707,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
this dressing has phosphoric acid in it all of the name brand ranch dressings do if you look even the H-E-B store brand ranch dressing here in south texas has it a lot of sodas have it to its not good for your teeth and if you are getting a bad taste in your mouth after eating this stuff its because of the phosphoric acid,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
this product is dangerous,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
very disappointing birthday present,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
what an icky trick!,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
